In [11]:
import os
from dotenv import load_dotenv
from taxi_demand_predictor.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
HOPSWORKS_PROJECT_NAME = os.environ['HOPSWORKS_PROJECT_NAME']

In [12]:
from datetime import datetime
import pandas as pd
from taxi_demand_predictor.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}...')

rides = pd.DataFrame()
for year in range(from_year, to_year + 1):
    rides_one_year = load_raw_data(year)
    rides = pd.concat([rides, rides_one_year])

File 2022-01 was already in local storage
File 2022-02 was already in local storage
File 2022-03 was already in local storage
File 2022-04 was already in local storage
File 2022-05 was already in local storage
File 2022-06 was already in local storage
File 2022-07 was already in local storage
File 2022-08 was already in local storage
File 2022-09 was already in local storage
File 2022-10 was already in local storage
File 2022-11 was already in local storage
File 2022-12 was already in local storage
File 2023-01 was already in local storage
File 2023-02 was already in local storage
File 2023-03 was already in local storage
File 2023-04 was already in local storage
File 2023-05 was already in local storage
2023-06 file is not available
2023-07 file is not available
2023-08 file is not available
2023-09 file is not available
2023-10 file is not available
2023-11 file is not available
2023-12 file is not available


In [13]:
print(f'{len(rides)}')

55840583


In [14]:
from taxi_demand_predictor.data import transform_raw_data_into_ts_data
ts_data = transform_raw_data_into_ts_data(rides)

See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
100%|██████████| 265/265 [00:02<00:00, 118.96it/s]


In [15]:
import hopsworks
print(hopsworks.__version__)
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

3.2.0
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/71842


In [16]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [17]:
FEATURE_GROUP_NAME = "time_series_hourly_feature_group"
FEATURE_GROUP_VERSION = 1

In [18]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key=['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

In [19]:
feature_group.insert(ts_data, write_options={'wait-for-job': False})

Uploading Dataframe: 0.00% |          | Rows 0/3281760 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_hourly_feature_group_1_offline_fg_backfill
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/71842/jobs/named/time_series_hourly_feature_group_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x142234ac0>, None)